In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import os
import sys
from multiprocessing import Pool, cpu_count
from tqdm.auto import tqdm
import pandas as pd

In [3]:
import warnings
warnings.filterwarnings('ignore', category=pd.errors.SettingWithCopyWarning)

In [4]:
project_root = os.path.abspath(
    os.path.join("C:/Users/jllgo/OneDrive/Documentos/UFMG/MSc & MSI 2/MatchSegmentation")
)
if project_root not in sys.path:
    sys.path.append(project_root)

In [5]:
from src.pass_networks.process_games import process_game
from src.pass_networks.process_games import load_game
from src.pass_networks.process_games import save_game
from src.pass_networks.process_match_info import get_match_info
from src.pass_networks.process_match_info import process_players
from src.pass_networks.process_intervals import get_interval_graphs
from src.pass_networks.process_intervals import load_graphs
from src.pass_networks.process_intervals import save_graphs

In [6]:
RAW_DATA_PATH = '../data/raw/PL-22-23'
INTERMEDIATE_DATA_PATH = '../data/intermediate/PL-22-23'
PROCESSED_DATA_PATH = "../data/processed/PL-22-23"
CSV_DATA_PATH = "../data/csv"

# 1. Get Data

In [7]:
games = os.listdir(RAW_DATA_PATH)
raw_game_ids = [game.split('.')[0] for game in games]
print(f"Games to process: {len(raw_game_ids)}")

Games to process: 190


In [8]:
pre_processed_games = os.listdir(INTERMEDIATE_DATA_PATH)
pre_processed_game_ids = [game.split('.')[0] for game in pre_processed_games]
print(f"Games already pre-processed: {len(pre_processed_game_ids)}")

Games already pre-processed: 81


In [9]:
processed = os.listdir(PROCESSED_DATA_PATH)
processed_game_ids = [game.split('.')[0] for game in processed]
print(f"Games already processed: {len(processed_game_ids)}")

Games already processed: 80


In [10]:
failed_ids = pd.read_csv(f"{CSV_DATA_PATH}/failed_matches.csv", header=None).values.astype(str).flatten().tolist()
failed_ids

['4437',
 '4445',
 '4448',
 '4458',
 '4446',
 '4493',
 '4499',
 '4502',
 '4509',
 '4510',
 '4524',
 '4527',
 '4528',
 '4529',
 '4530',
 '4531',
 '4532',
 '4533',
 '4534',
 '4535',
 '4536',
 '4537',
 '4538',
 '4539',
 '4540',
 '4541',
 '4542',
 '4543',
 '4544',
 '4545',
 '4546',
 '4547',
 '4548',
 '4549',
 '4550',
 '4551',
 '4552',
 '4553',
 '4554',
 '4555',
 '4556']

In [11]:
pre_processed_games = [game_id for game_id in pre_processed_game_ids if game_id not in failed_ids]
print(f"Pre Processed Games to Process: {len(pre_processed_games)}")

Pre Processed Games to Process: 80


In [12]:
matches_to_process = [game_id for game_id in raw_game_ids if game_id not in pre_processed_game_ids and game_id not in failed_ids][:20]
print(f"Games to process: {len(matches_to_process)}")

Games to process: 20


In [13]:
matches = []

failed_matches = []

# #Check if the game has already been pre-processed
# if len(pre_processed_game_ids) > 0:
#     tasks = [(INTERMEDIATE_DATA_PATH,game_id) for game_id in pre_processed_game_ids]

#     for task in tqdm(tasks, desc="Loading Pre-Processed Games", total=len(tasks)):
#         metadata_df, players_df, events_df = load_game(task)
#         matches.append((task[1],metadata_df, players_df, events_df))

if len(matches_to_process) > 0:

    for game_id in tqdm(matches_to_process, desc="Processing Games", total=len(matches_to_process)):
        metadata_df, players_df, events_df = process_game((RAW_DATA_PATH, game_id))
        if metadata_df.empty or players_df.empty or events_df.empty:
            failed_matches.append(game_id)
            continue
        else:
            matches.append((game_id,metadata_df, players_df, events_df))
            save_game(metadata_df,players_df,events_df,INTERMEDIATE_DATA_PATH,game_id)



Processing Games:   0%|          | 0/20 [00:00<?, ?it/s]

Loading frames from match 4557: 0 frames [00:00, ? frames/s]

Validating frames:   0%|          | 0/174886 [00:00<?, ? frames/s]

Loading frames from match 4558: 0 frames [00:00, ? frames/s]

Validating frames:   0%|          | 0/174139 [00:00<?, ? frames/s]

...

Error processing match_id 4574: cannot access local variable 'row' where it is not associated with a value


Loading frames from match 4575: 0 frames [00:00, ? frames/s]

Validating frames:   0%|          | 0/155080 [00:00<?, ? frames/s]

Loading frames from match 4576: 0 frames [00:00, ? frames/s]

Validating frames:   0%|          | 0/177235 [00:00<?, ? frames/s]

In [14]:
failed_matches = pd.DataFrame(failed_matches, columns=["game_id"])
failed_matches.to_csv(f"{CSV_DATA_PATH}/failed_matches.csv", index=False, header=False, mode='a')

In [15]:
players_info, teams_info, games_info = get_match_info(CSV_DATA_PATH)

# 2. Data Treatment

In [16]:
players_dfs = []
events_dfs = []
metadata_dfs = []
match_ids = []

for match_dfs in tqdm(matches, desc="Data Processing"):
    match_id, metadata_df, players_df, events_df = match_dfs
    players_match_info = players_info[players_info['match_id'] == int(match_id)].reset_index(drop=True)
    match_info = games_info[games_info['match_id'] == int(match_id)].reset_index(drop=True)

    players_df = process_players(players_df, match_info, players_match_info, metadata_df)

    players_dfs.append(players_df)
    events_dfs.append(events_df)
    metadata_dfs.append(metadata_df)
    match_ids.append(match_id)

players_df = pd.concat(players_dfs)
events_df = pd.concat(events_dfs)
metadata_df = pd.concat(metadata_dfs)

Data Processing:   0%|          | 0/15 [00:00<?, ?it/s]

In [17]:
players_interval_df = players_df.groupby(['match_id','interval_id','team_id','player_id','nickname','shirt']).agg(
    x=('x', 'mean'),
    y=('y', 'mean')
).reset_index()

players_interval_df['player_id'] = players_interval_df['player_id'].astype(int)
players_interval_df['match_id'] = players_interval_df['match_id'].astype(int)
players_interval_df['interval_id'] = players_interval_df['interval_id'].astype(int)
players_interval_df['team_id'] = players_interval_df['team_id'].astype(int)
players_interval_df['shirt'] = players_interval_df['shirt'].astype(int)

players_interval_df[players_interval_df['interval_id']==1]

,match_id,interval_id,team_id,player_id,nickname,shirt,x,y
0,4557,1,6,25,Raheem Sterling,17,7.028867,19.076528
1,4557,1,6,88,Kepa,1,-37.909179,2.180471
2,4557,1,6,96,César Azpilicueta,28,-5.929470,-16.629494
3,4557,1,6,98,Jorginho,5,-3.976327,2.191415
4,4557,1,6,101,Ruben Loftus-Cheek,12,-3.226055,13.204679
...,...,...,...,...,...,...,...,...
15266,4576,1,12,1947,Donny van de Beek,34,-17.649229,-1.406180
15267,4576,1,12,6020,Cristiano Ronaldo,7,-26.218487,3.445821
15268,4576,1,12,7443,Christian Eriksen,14,-7.186674,-6.382556
15269,4576,1,12,7832,Alejandro Garnacho,49,-17.719599,-16.179115


In [18]:
match_intervals = players_df[['match_id','interval_id','team_id','team']].drop_duplicates().reset_index(drop=True)
match_intervals = match_intervals[match_intervals['interval_id'].notna()].reset_index(drop=True)
match_intervals

,match_id,interval_id,team_id,team
0,4557,1.0,6,home
1,4557,1.0,12,away
2,4557,2.0,6,home
3,4557,2.0,12,away
4,4557,3.0,6,home
...,...,...,...,...
1477,4576,48.0,12,away
1478,4576,49.0,3,home
1479,4576,49.0,12,away
1480,4576,50.0,3,home


In [19]:
data = [{
    'match_id':row['match_id'],
    'interval_id':row['interval_id'],
    'team_id': row['team_id'],
    'player_id': -1,
    'nickname': 'Goal',
    'shirt': -1,
    'x': 52 if row['team']=='home' else -52,
    'y': 0
} for _,row in match_intervals.iterrows()]

data = pd.DataFrame(data)

players_interval_df = pd.concat([players_interval_df, data], ignore_index=True).sort_values(['match_id','interval_id','team_id','player_id']).reset_index(drop=True)
players_interval_df

,match_id,interval_id,team_id,player_id,nickname,shirt,x,y
0,4557,1.0,6,-1,Goal,-1,52.000000,0.000000
1,4557,1.0,6,25,Raheem Sterling,17,7.028867,19.076528
2,4557,1.0,6,88,Kepa,1,-37.909179,2.180471
3,4557,1.0,6,96,César Azpilicueta,28,-5.929470,-16.629494
4,4557,1.0,6,98,Jorginho,5,-3.976327,2.191415
...,...,...,...,...,...,...,...,...
17830,4576,50.0,12,3112,Anthony Elanga,36,-46.793533,12.937347
17831,4576,50.0,12,6020,Cristiano Ronaldo,7,-44.349994,1.708914
17832,4576,50.0,12,7443,Christian Eriksen,14,-38.159522,10.114210
17833,4576,50.0,12,8023,Lisandro Martinez,6,-22.225844,-4.234964


In [20]:
players_interval_df = players_interval_df[players_interval_df['interval_id'].notna()].reset_index(drop=True)
players_interval_df

,match_id,interval_id,team_id,player_id,nickname,shirt,x,y
0,4557,1.0,6,-1,Goal,-1,52.000000,0.000000
1,4557,1.0,6,25,Raheem Sterling,17,7.028867,19.076528
2,4557,1.0,6,88,Kepa,1,-37.909179,2.180471
3,4557,1.0,6,96,César Azpilicueta,28,-5.929470,-16.629494
4,4557,1.0,6,98,Jorginho,5,-3.976327,2.191415
...,...,...,...,...,...,...,...,...
17830,4576,50.0,12,3112,Anthony Elanga,36,-46.793533,12.937347
17831,4576,50.0,12,6020,Cristiano Ronaldo,7,-44.349994,1.708914
17832,4576,50.0,12,7443,Christian Eriksen,14,-38.159522,10.114210
17833,4576,50.0,12,8023,Lisandro Martinez,6,-22.225844,-4.234964


### Events

In [21]:
pass_df = events_df[(events_df['possession_type']=='PASS') & (events_df['outcome']=='C')][['match_id','player_id','receiver_id','event_id','possession_id','team_id']]
pass_df.head()

,match_id,player_id,receiver_id,event_id,possession_id,team_id
0,4557,133,1683,5962679,5821803,12
1,4557,1683,8023,5962682,5821806,12
2,4557,8023,1675,5962688,5821812,12
3,4557,1675,8023,5962693,5821816,12
4,4557,8023,1675,5962697,5821820,12


In [22]:
carry_df = events_df[(events_df['possession_type']=='CARRY') & ((events_df['carry_type'].isin(['T','C'])) | ((events_df['carry_type']=='D')&(events_df['outcome'].isin(['K','B']))))][['match_id','player_id','receiver_id','event_id','possession_id','team_id']]
carry_df.head()

,match_id,player_id,receiver_id,event_id,possession_id,team_id
50,4557,25,25,5962983,5822083,6
111,4557,96,96,5963264,5833872,6
156,4557,1675,1675,5963459,5822548,12
199,4557,101,101,5963643,5822723,6
242,4557,1331,1331,5963767,5822849,12


In [23]:
shots_df = events_df[(events_df['possession_type']=='SHOT')][['match_id','player_id','receiver_id','event_id','possession_id','team_id']]
shots_df.head()

,match_id,player_id,receiver_id,event_id,possession_id,team_id
94,4557,123,-1,5963195,5822284,12
138,4557,8039,-1,5963337,5822424,12
144,4557,5154,-1,5963421,5822507,6
256,4557,136,-1,5963845,5822923,12
295,4557,136,-1,5964096,5823161,12


In [24]:
network_events_df = pd.concat([pass_df,carry_df, shots_df]).reset_index(drop=True)
network_events_df.head()

,match_id,player_id,receiver_id,event_id,possession_id,team_id
0,4557,133,1683,5962679,5821803,12
1,4557,1683,8023,5962682,5821806,12
2,4557,8023,1675,5962688,5821812,12
3,4557,1675,8023,5962693,5821816,12
4,4557,8023,1675,5962697,5821820,12


### Networks

In [25]:
network_events_df['match_id'] = network_events_df['match_id'].astype(float)
network_events_df['event_id'] = network_events_df['event_id'].astype(float)
network_events_df['possession_id'] = network_events_df['possession_id'].astype(float)
metadata_df['match_id'] = metadata_df['match_id'].astype(float)
metadata_df['event_id'] = metadata_df['event_id'].astype(float)
metadata_df['possession_id'] = metadata_df['possession_id'].astype(float)

network_frame_df = network_events_df.merge(metadata_df[['match_id','interval_id','frame_id','event_id','possession_id']], on=['match_id','event_id','possession_id'], how='left')
network_frame_df.head()

,match_id,player_id,receiver_id,event_id,possession_id,team_id,interval_id,frame_id
0,4557.0,133,1683,5962679.0,5821803.0,12,1.0,417.0
1,4557.0,1683,8023,5962682.0,5821806.0,12,1.0,460.0
2,4557.0,8023,1675,5962688.0,5821812.0,12,1.0,534.0
3,4557.0,1675,8023,5962693.0,5821816.0,12,1.0,627.0
4,4557.0,8023,1675,5962697.0,5821820.0,12,1.0,717.0


In [26]:
# Group by match_id, interval_id, team_id, player_id, receiver_id to get the number of interactions between players

grouped_df = network_frame_df.groupby(['match_id','interval_id','team_id','player_id','receiver_id']).size().reset_index(name='count')
grouped_df['match_id']=grouped_df['match_id'].astype(int)
grouped_df['interval_id']=grouped_df['interval_id'].astype(int)
grouped_df['team_id']=grouped_df['team_id'].astype(int)
grouped_df['player_id']=grouped_df['player_id'].astype(int)
grouped_df['receiver_id']=grouped_df['receiver_id'].astype(int)

grouped_df.head()

,match_id,interval_id,team_id,player_id,receiver_id,count
0,4557,1,12,114,8023,1
1,4557,1,12,123,1331,1
2,4557,1,12,123,8023,1
3,4557,1,12,133,114,1
4,4557,1,12,133,1683,1


In [27]:
grouped_df = grouped_df.merge(players_info[['match_id','player_id','nickname','shirt_number']], how='left', left_on=['match_id','player_id'], right_on=['match_id','player_id']).drop_duplicates().reset_index(drop=True)
grouped_df = grouped_df.merge(players_info[['match_id','player_id','nickname','shirt_number']], how='left', left_on=['match_id','receiver_id'], right_on=['match_id','player_id']).drop_duplicates().reset_index(drop=True)

grouped_df = grouped_df.rename(columns={'player_id_x':'player_id','nickname_x':'player_nickname','shirt_number_x':'player_shirt','nickname_y':'receiver_nickname','shirt_number_y':'receiver_shirt'})
grouped_df = grouped_df.drop(['player_id_y'], axis=1)

grouped_df['receiver_nickname'] = grouped_df['receiver_nickname'].fillna('Goal')
grouped_df['receiver_shirt'] = grouped_df['receiver_shirt'].fillna(-1)
grouped_df['receiver_shirt'] = grouped_df['receiver_shirt'].astype(int)

grouped_df.head()

,match_id,interval_id,team_id,player_id,receiver_id,count,player_nickname,player_shirt,receiver_nickname,receiver_shirt
0,4557,1,12,114,8023,1,David de Gea,1,Lisandro Martinez,6
1,4557,1,12,123,1331,1,Luke Shaw,23,Jadon Sancho,25
2,4557,1,12,123,8023,1,Luke Shaw,23,Lisandro Martinez,6
3,4557,1,12,133,114,1,Bruno Fernandes,8,David de Gea,1
4,4557,1,12,133,1683,1,Bruno Fernandes,8,Casemiro,18


# Graphs

In [28]:
graph_games = os.listdir(PROCESSED_DATA_PATH)
graph_game_ids = [game.split('.')[0] for game in graph_games]
print(f"Games to process: {len(graph_game_ids)}")

Games to process: 80


In [29]:
pre_processed_ids = [game_id for game_id in match_ids if game_id not in graph_game_ids]
print(f"Games to process graphs: {len(pre_processed_ids)}")

Games to process graphs: 15


In [30]:
#graphs = []

# if len(graph_game_ids) > 0:

#     for game_id in tqdm(graph_game_ids, desc="Loading Graphs", total=len(graph_game_ids)):
#         match_graphs = load_graphs(game_id, PROCESSED_DATA_PATH)
#         graphs.append((task[1],match_graphs))

if len(pre_processed_ids) > 0:

    for game_id in tqdm(pre_processed_ids, desc="Processing Graphs", total=len(pre_processed_ids)):

        match_grouped_df = grouped_df[grouped_df['match_id']==int(game_id)].reset_index(drop=True)
        match_players_interval_df = players_interval_df[players_interval_df['match_id']==int(game_id)].reset_index(drop=True)

        match_graphs = get_interval_graphs(match_grouped_df, match_players_interval_df)
        save_graphs(game_id, PROCESSED_DATA_PATH, match_graphs)
        #graphs.append((game_id,match_graphs))

Processing Graphs:   0%|          | 0/15 [00:00<?, ?it/s]

Processing intervals:   0%|          | 0/48 [00:00<?, ?it/s]

Processing intervals:   0%|          | 0/48 [00:00<?, ?it/s]

Processing intervals:   0%|          | 0/49 [00:00<?, ?it/s]

Processing intervals:   0%|          | 0/50 [00:00<?, ?it/s]

...

Processing intervals:   0%|          | 0/49 [00:00<?, ?it/s]

Processing intervals:   0%|          | 0/48 [00:00<?, ?it/s]

Processing intervals:   0%|          | 0/50 [00:00<?, ?it/s]

Processing intervals:   0%|          | 0/50 [00:00<?, ?it/s]

Processing intervals:   0%|          | 0/50 [00:00<?, ?it/s]